In [23]:
# Import appropriate packages to clean and analyse the data
import pandas as pd # To handle dataframes
import plotly.express as px # To visualize data
import numpy as np 
import matplotlib.pyplot as plt 
import calendar # To convert month names
import streamlit as st # To upload streamlit objects
from geopy.geocoders import Nominatim # For finding locations
import pycountry # for grouping countries into continents

In [24]:
df = pd.read_csv('./data/df_after_locations_merge.csv', parse_dates=['reported_date','encounter_date'])

In [25]:
#df should
df.isnull().sum()

id                   0
encounter_date       0
reported_date        0
year                 0
month                0
hour                 0
season               0
country_code       258
country             68
region             370
locale             314
latitude             0
longitude            0
ufo_shape         1930
duration_secs        0
description         15
dtype: int64

### 2.3.3 Check Description for Insights

The second process will look to the descriptions to see if missing data can be added. This will be done for all location columns, and also the ufo shape column. Do this using a five step process:

1. Create a new dataframe with null values.

2. Create a list to compare the unique values to.

3. Compare the two sets using the set() method.

4. Determine which values should be integrated.

5. Merge appropriate values.

**Note:** If more time was granted, each description piece would be read to allow for values that might not yet exist in the unique values set.

#### 2.3.3.1 Create Functions

In [26]:
# Create description_match function that does the first 3 steps
def description_match(dataframe, column_name):
    # 1. Create null dataframe
    null_df= dataframe[dataframe[column_name].isna()]
    #drop where no description exists
    null_df= null_df.dropna(subset='description')
    
    # Make description lower case for easier searching 
    df['description']= df['description'].str.lower()
    
    # 2. Create unique list from column to compare to
    unique_list= np.array(dataframe[column_name].unique()).astype(str)
    #make np.array list lower case for easier searching
    unique_list= np.char.lower(unique_list)

    # 3. Compare both sets
    # Create set list from unique list to be able to compare intersection
    list_set = set(unique_list)
    
    # Create matches list that will contain matched data
    
    match = []
    
    # Create string set by splitting words from description
    for i in range(len(null_df)): 
        string_set = set(null_df.iloc[i,15].split())

        # Where one word matches, add index, match word and description to matches list
        if len(list_set.intersection(string_set))==1:
            match.append([null_df.iloc[i,0], list(list_set.intersection(string_set))[0],np.nan,null_df.iloc[i,15]])
        
        # Where two words match, add index, both match words, and description
        if len(list_set.intersection(string_set))== 2:
            match.append([null_df.iloc[i,0], list(list_set.intersection(string_set))[0], list(list_set.intersection(string_set))[1],null_df.iloc[i,15]])
            
    # Save list as dataframe
    potential_matches= pd.DataFrame(match)
    
    # Rename column names
    potential_matches= potential_matches.rename(columns={0:'id', 1:column_name, 2:column_name+'_2', 3:'description'})

    # Change column types to string
    potential_matches[column_name]= potential_matches[column_name].astype(str)
    potential_matches[column_name+'_2']= potential_matches[column_name+'_2'].astype(str)
    
    # Captialize string columns
    potential_matches[column_name] = potential_matches[column_name].apply(lambda x: x.capitalize())
    potential_matches[column_name+'_2'] = potential_matches[column_name+'_2'].apply(lambda x: x.capitalize())
    
    # Return potential_matches
    return potential_matches

In [27]:
#4. Determine which values should be integrated
def drop_edit_rows(potential_matches, rows_to_drop, rows_to_edit=[], new_string=[]):
    
    #edit rows
    for i in range(len(rows_to_edit)):
        potential_matches.iloc[rows_to_edit[i],1] = new_string[i]
    
    #drop rows
    matches = potential_matches.drop(labels= rows_to_drop)

    return matches

In [28]:
# 5 Merge appropriate values
def merge_matches(dataframe ,matches, column_name):
    # Print sum of missing values to compare before
    print('Number of Nan values before merge = ', dataframe[column_name].isna().sum())
    
    #for i in range(len(matches)):
        #dataframe.iloc[matches.iloc[i,0], dataframe.columns.get_loc(column_name)]= matches.iloc[i,1]
    dataframe[column_name] = dataframe[column_name].combine_first(matches[column_name])


    combined_matches = dataframe
        
    # Print sum of missing values again to compare after merge
    print('Number of Nan values after merge = ', combined_matches[column_name].isna().sum())
    
    return combined_matches

#### 2.3.3.2 Countries

In [29]:
# Steps 1-3
# Allow for larger width to read description 
pd.options.display.max_colwidth = 1000

# Run description match on country column
country_matches= description_match(df, 'country')

# Print country_matches
country_matches

,id,country,country_2,description
0,26926,Japan,Nan,i was sitting in seat 47k (a window seat on the right side of the jet airliner) of japan airlines flight jl 060 on feb 16 2006 on my
1,39914,Mexico,Nan,three objects in the sky in the pacific ocean off the coast of mexico or usa on a cruise ship.


In [30]:
df.iloc[country_matches.iloc[0,0], df.columns.get_loc('country')]

'United States'

In [31]:
country_matches.iloc[0,0]

26926

In [32]:
df.columns.get_loc('country')

8

In [33]:
df.iloc[26926, 8]

'United States'

In [34]:
df.iloc[26926,df.columns.get_loc('country')]

'United States'

**Step 4**:
As each description clearly relates to a country location, run merge_matches function.

In [35]:
# 5. Merge appropriate values
# Run merge_match on all matches from above
df= merge_matches(df, country_matches, 'country')

Number of Nan values before merge =  68
Number of Nan values after merge =  68


In [36]:
df.iloc[26926,8] = 'Japan'
print(df.iloc[26926, 8])
df[df['id']==26926]

Japan


,id,encounter_date,reported_date,year,month,hour,season,country_code,country,region,locale,latitude,longitude,ufo_shape,duration_secs,description
26909,26926,2006-02-16,2006-07-16,2006,February,0,Winter,NaN,NaN,NaN,NaN,-8.783195,-124.508523,Triangle,30.0,i was sitting in seat 47k (a window seat on the right side of the jet airliner) of japan airlines flight jl 060 on feb 16 2006 on my


In [37]:
#df['country'] = df['country'].combine_first(country_matches['country']) # this still doesnt work
for i in range(len(country_matches)):
    df.iloc[country_matches.iloc[i,0], df.columns.get_loc('country')]= country_matches.iloc[i,1]
    

In [38]:
df['country'].isnull().sum()

68

In [39]:
df[df['id']==26926]

,id,encounter_date,reported_date,year,month,hour,season,country_code,country,region,locale,latitude,longitude,ufo_shape,duration_secs,description
26909,26926,2006-02-16,2006-07-16,2006,February,0,Winter,NaN,NaN,NaN,NaN,-8.783195,-124.508523,Triangle,30.0,i was sitting in seat 47k (a window seat on the right side of the jet airliner) of japan airlines flight jl 060 on feb 16 2006 on my


In [40]:
# As country data is now complete as possible, link missing country codes
# Save country_codes
country_codes=df[['country_code','country']]

# Group by 'country' and fill missing values in 'country_code' with the mode of each group
filled_country_codes = country_codes.copy()  # Create a copy of the dataFrame to avoid copyerror
filled_country_codes['country_code'] = country_codes.groupby('country')['country_code'].transform(lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else np.nan))

# Merge the filled_country_codes DataFrame back to the original DataFrame based on the index
df['country_code'] = filled_country_codes['country_code']
     

#### 2.3.3.3 Regions

In [41]:
# Steps 1-3
# Run description match on region column
potential_region=description_match(df, 'region')
potential_region

,id,region,region_2,description
0,12980,Bali,Nan,silent ufo hanging over the ocean bali
1,19056,Aegean,Nan,1/2/2000 aegean sea
2,25137,Southeast,Nan,two brothers witness 10 to 15 spherical objects traveling southeast toward tokyo.
3,53934,Centre,Nan,three distinct glowing orbs over the sony centre berlin. brilliant white in colour daylight other events night time. three people saw t
4,60108,Cebu,Nan,i am looking at the clear skies near near the airplane path from davao city to cebu city. i was at my verandah in my home when i saw t
5,63181,Florida,Nan,july 6 2007 aboard the carnival liberty atlantic ocean south of florida keys observed three round blue/green objects.
6,68274,Centre,Nan,three light spots converging to centre and disappearing at amazing speed


**Step 4:** Some rows are problematic and need to be deleted. For instance England is a country, not a region, the same goes for "centre" in row 15. Others need to be editted to reflect their true region such as rows 6, 7, 9, and 15. Information is saved in lists below.

In [45]:
# Save lists to edit potential_matches dataframe
# rows to drop
drop_regions= [6]

#regions to edit
edit_regions= [2, 3]

#strings to edit regions
new_string_regions = ['Kanto', 'Berlin']

In [46]:
# Pass through the drop_edit_rows function
region_matches= drop_edit_rows(potential_region, drop_regions, edit_regions, new_string_regions)

# print result
region_matches

,id,region,region_2,description
0,12980,Bali,Nan,silent ufo hanging over the ocean bali
1,19056,Aegean,Nan,1/2/2000 aegean sea
2,25137,Kanto,Nan,two brothers witness 10 to 15 spherical objects traveling southeast toward tokyo.
3,53934,Berlin,Nan,three distinct glowing orbs over the sony centre berlin. brilliant white in colour daylight other events night time. three people saw t
4,60108,Cebu,Nan,i am looking at the clear skies near near the airplane path from davao city to cebu city. i was at my verandah in my home when i saw t
5,63181,Florida,Nan,july 6 2007 aboard the carnival liberty atlantic ocean south of florida keys observed three round blue/green objects.


In [49]:
# 5. Merge appropriate values

# Run merge_match on all matches from above
df= merge_matches(df, region_matches, 'region')

Number of Nan values before merge =  370
Number of Nan values after merge =  370


#### 2.3.3.4 Locale

In [50]:
# Steps 1-3
# Run description match on locale column
locale_matches= description_match(df, 'locale')

# Show results
locale_matches

,id,locale,locale_2,description
0,515,Bright,Nan,bright object seemingly appeared out of nowhere in the indian ocean miles from any land.
1,1882,White,Nan,october 1990 - saudi arabia - large dark triangular object - 3 white lights - 20 seconds - no sound - no stars behind object
2,4013,Macedonia,Nan,report from macedonia
3,4212,Blue,Nan,blue colour sphere was obsereved from containershipdia abt 4mtrsfrom dist of abt 8mtr.moved after twds italian coast.
4,4274,Bright,Nan,bright flash lightens up night sky object fires across sky at unbelievable speed&#8230; ((nuforc note: possible meteor. pd))
...,...,...,...,...
118,74114,Trail,Nan,a flash movin slow at firstchanges into a spherical shapecirclesraces westleavin behind a hazy greenish smoke trail
119,74303,Orange,Nan,flashing object changes colour . second obect- orange sphere does nothing except sit there.
120,76278,Bow,Nan,the light clearly lit up the bow of the vessel where no light should have been in the middle of the atlantic.
121,76932,Center,Nan,ship object was landed at the center of tol road approx 60 km from jakarta to cikampek in dawuan place


**Step 4:** As ample locale names are supplied, many irrelevant matches arise. For instance, Where the light has been described as bright, a match exists. Filter these out.

In [51]:
locale_matches= locale_matches[(locale_matches['locale'] != 'Bright') & (locale_matches['locale'] != 'Blue')
                               & (locale_matches['locale_2'].isna()==False) & (locale_matches['locale'] != 'White')
                               & (locale_matches['locale'] != 'Edge') & (locale_matches['locale'] != 'Center')
                               & (locale_matches['locale'] != 'Bow') & (locale_matches['locale'] != 'Oblong')
                               & (locale_matches['locale'] != 'Tiny') & (locale_matches['locale'] != 'West')
                               & (locale_matches['locale'] != 'Orange') & (locale_matches['locale'] != 'Star')]

In [52]:
# Reset index
locale_matches= locale_matches.reset_index()

In [53]:
# Drop index column
locale_matches= locale_matches.drop(columns='index')

##print all results using .to_string to view in text editor for deeper analysis
print(locale_matches.to_string())

       id      locale  locale_2                                                                                                                                  description
0    4013   Macedonia       Nan                                                                                                                        report from macedonia
1    4292        Hull       Nan                                                               3 red lights flying fast in loose triangular formation hull east yorkshire uk.
2   11333    Brisbane       Nan                                                                    3 disc like crafts sighted on the eve of 2013 in brisbane area australia.
3   14770        Lake    Bright                             at the approx time of 8.15pm a bright circular red light was spotted north east of lake macquarie nsw australia.
4   17450       Trail      Blue        starfox shaped craft with blue glow and smoke/debris trail behind with no sound within 2500m and

In [54]:
# Save lists to edit potential matches dataframe
# rows to drop
drop_locales= [9, 10, 12, 14, 15, 18, 20, 21, 22, 23, 25, 29, 31, 32, 33, 35, 36, 38, 41, 43]

#locales to edit
edit_locales= [3, 6, 16, 17, 19, 24, 39, 40]

#strings to edit locales
new_string_locales = ['Macquarie Park', 'Hull', 'Yarmouth', 'Huntington', 'Essex', 'Cozumel', 'Atlantic', 'Surrey']

In [55]:
# Pass through the drop_edit_rows function
locale_matches= drop_edit_rows(locale_matches, drop_locales, edit_locales, new_string_locales)

# print result
locale_matches

,id,locale,locale_2,description
0,4013,Macedonia,Nan,report from macedonia
1,4292,Hull,Nan,3 red lights flying fast in loose triangular formation hull east yorkshire uk.
2,11333,Brisbane,Nan,3 disc like crafts sighted on the eve of 2013 in brisbane area australia.
3,14770,Macquarie Park,Bright,at the approx time of 8.15pm a bright circular red light was spotted north east of lake macquarie nsw australia.
4,17450,Trail,Blue,starfox shaped craft with blue glow and smoke/debris trail behind with no sound within 2500m and an altitude no higher than 400 feet.
5,18592,Atlantic,Nan,crew of lear jet over the atlantic ocean report a bizarre red strobing object.
6,20065,Hull,Nan,anomalous object with erractic flight patterns over lethbridge alberta.
7,20878,England,Hull,three orange/amber lights observed in hull england on the 27th december 2007 at 21:25
8,22674,Maiden,Nan,strange object/light over maiden nc area
11,23782,Diamond,Nan,diamond


In [57]:
# 5.Merge appropriate values
# Run merge_match on all matches from above
df= merge_matches(df, locale_matches, 'locale')

Number of Nan values before merge =  314
Number of Nan values after merge =  314


#### 2.3.3.5 UFO Shape

In [58]:
# Steps 1-3
# Run description match on ufo_shape column
ufo_matches= description_match(df, 'ufo_shape')

#print all results using .to_string to view in text editor for deeper analysis
print(ufo_matches.to_string())

**Step 4:** Taking a quick look, description words are not always representative. The biggest culptrit is those of shape "Other". Cross has also been used inaccurately. Drop these.

In [ ]:
# Find ufo_shape values where other or cross has been selected
ufo_other= ufo_matches[ufo_matches['ufo_shape']=='Other']['id']
ufo_cross= ufo_matches[ufo_matches['ufo_shape']=='Cross']['id']

# Combine other and cross values, find index values
drop_ufo = pd.concat([ufo_cross, ufo_other]).index

In [ ]:
# Pass through the drop_edit_rows function
drop_edit_rows(ufo_matches, drop_ufo)

In [ ]:
# 5.Merge appropriate values
# Run merge_match on all matches from above
df= merge_matches(ufo_matches, 'ufo_shape')

#### 2.3.3.6 Null Values Summary

Using the coordinate and description methods has lead to a decrease in null values of over 1000 values. Missing data for each column is less than 0.5%, except ufo_shape which has 1.6% of values missing.


In [ ]:
#Percentage of values that are null
round(df.isnull().sum()/len(df),3)*100

### 2.3.4 Duplicates

Each entry is treated with an individual ID, so duplicates on every metric will be zero. However, duplicates may be found if two of the same events are reported. The chances that the exact string description and the local area it is reported in is the exact same is low. Thus, duplicates will be dropped on these metrics.

In [ ]:
# Show all duplicates
print('Duplicates on all metrics= ',df.duplicated().sum())

# Print the length of duplicated rows based on description and locale
print('Duplicates on description and locale= ',len(df[df.duplicated(['description','locale'])]))

# Drop these rows
df= df.drop_duplicates(['description','locale'])

### 2.3.5 Add Useful Data

Data which will be useful includes data on time, ufo color and continent in which the encounter is reported.

#### 2.3.5.1 Time Data
The duration column currently accounts for encounters based on seconds. Create new columns to see how duration works on a minutely, hourly and daily basis. Create an age category based on how long ago the encounter was reported. Also create a column which shows the difference between when the encounter was reported and when the encounter actually occurred.

In [ ]:
# Create duration in minutes column
df['duration_mins'] = df.loc[:,'duration_secs'] / 60

# Create duration in hours
df['duration_hours'] = df['duration_mins'] / 60

# Create duration in days 
df['duration_days'] = df['duration_hours'] / 24

In [ ]:
# Create age column which calculates how many years have passed since it was reported
df['age'] = 2023 - (df['encounter_date'].dt.year + (df['encounter_date'].dt.month/12))

In [ ]:
# Create reported difference column which is the difference in years between the reported and encountered dates
df['reported_diff'] = round((df['reported_date'].dt.year + df['reported_date'].dt.month/12)
                            - (df['encounter_date'].dt.year + df['encounter_date'].dt.month/12), 2)

#### 2.3.5.2 Color Data
Add data which assigns a color based on the encounter. 

In [ ]:
# Create list of color
colors={'white','yellow','orange','red','green','blue','purple','brown','silver','gold','gray','grey','black', 'amber', 'aqua','indigo','pink'}

# Create ufo_color column
df['ufo_color'] = np.nan


In [ ]:
df.iloc[:,16]

In [ ]:

# Create loop to determine if details column provides insight into ufo colors
for i in range(len(df)): 
    if df.iloc[i,15] is not np.nan: # Where description is not null
        string_set = set(df.iloc[i,15].split()) # Split description
        if len(colors.intersection(string_set))==1: # where there is only one color:
            matched_color = list(colors.intersection(string_set))[0] # Let the color be the first mentioned color
            if matched_color not in (df.iloc[i,8:11]).values: # Ensure that the matched color is not referring to the country, region, or locale instead
                df.iloc[i,21] = matched_color
                
        if len(colors.intersection(string_set)) > 1: # where there are multiple colors
            if matched_color not in (df.iloc[i,8:11]).values: # Ensure that the matched color is not referring to the country, region, or locale instead
                df.iloc[i,21] = 'multicolor'
                
    

In [ ]:
# Consolidate Gray Colors
df['ufo_color'] = df['ufo_color'].replace('grey','gray')

In [ ]:
# Capitalize color column
df['ufo_color']= df['ufo_color'].str.capitalize()

In [ ]:
# Percentage of each color
round(df['ufo_color'].value_counts(dropna=False)/len(df),4)*100

**Note:** As much of this data is still missing, this data will only supplement our analysis. Also this assumes that if a color is mentioned it is in reference to the ufo color. However, these colors should not be referencing the location name which has been accounted for.

#### 2.3.5.3 Continent Data

Create list of countries in their respective continents


In [ ]:
country_unique= df['country'].unique()

country_code_unique=df['country_code'].unique()

print(len(country_code_unique))
len(country_unique)


In [ ]:
country_continents= pd.DataFrame([country_code_unique,country_unique])
country_continents

## 2 Analysis

The analysis will 

In [ ]:
# Order months
months_ordered = ['January', 'February', 'March', 'April','May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

# Create scatter with plot.ly, 
fig = px.scatter( df, x='year', y= 'duration_secs', color='month', hover_data='ufo_shape', \

                category_orders={'month': months_ordered})

# Name axis titles and title
fig.update_layout(xaxis_title='Year', yaxis_title=('Time (secs)'), 
                  title= '',
                 legend_title = 'Month')

# Show figure
fig.show()

In [ ]:
df2 = df[df['duration_hours'] < 1]

In [ ]:
# Create histogram with plot.ly, set x to price and color to paint color, create discrete color map to match colors
fig = px.histogram(df2, x='duration_mins', color = 'ufo_shape', category_orders={'month': months_ordered})

# Rename axis, title and legend
fig.update_layout(xaxis_title='Duration of Encounter (secs)', yaxis_title=('Count'), title= '',
                 legend_title = 'Color')



# Set maximim axes to 50000
fig.update_xaxes(maxallowed=50000)

# Show figure
fig.show()

In [ ]:
# Find unique shapes
shapes= df['ufo_shape'].unique()

# Create 
shapes_for_map = st.selectbox('Shape of the sighted UFO:', shapes)

# Create variables to measure by for map
list_for_map= ['duration_secs','age'] 
choice_for_map= st.selectbox('Increase bubble size by:', list_for_map)

In [ ]:
import plotly.graph_objects as go

scl = [0,"rgb(150,0,90)"],[0.125,"rgb(0, 0, 200)"],[0.25,"rgb(0, 25, 255)"],\
[0.375,"rgb(0, 152, 255)"],[0.5,"rgb(44, 255, 150)"],[0.625,"rgb(151, 255, 0)"],\
[0.75,"rgb(255, 234, 0)"],[0.875,"rgb(255, 111, 0)"],[1,"rgb(255, 0, 0)"]

df1 = df[df['ufo_shape'] == shapes_for_map]



fig2 = go.Figure(data=go.Scattergeo(
    lat = df1['latitude'],
    lon = df1['longitude'],
    text = df1['description'],
    hovertext= df1['description'],
    marker = dict(
        color = df1['age'],
        colorscale = scl,
        reversescale = True,
        opacity = 0.7,
        size = 5,
        colorbar = dict(
            titleside = "top",
            outlinecolor = "rgba(68, 68, 68, 0)",
            title= 'Age'
        )
    )
))


fig.update_layout(
    geo = dict(
        scope = 'world',
        showland = True,
        landcolor = "rgb(212, 212, 212)",
        subunitcolor = "rgb(255, 255, 255)",
        countrycolor = "rgb(255, 255, 255)",
        showlakes = True,
        lakecolor = "rgb(255, 255, 255)",
        showsubunits = True,
        showcountries = True,
        resolution = 110
    ),
    title='Encounters Where the UFO Shape is a {}'.format(shapes_for_map), 
    width=1000,
    height=800,
    legend_title='Age'

)
fig2.show()

In [ ]:
c_d = df[df['country'] == 'Bermuda']['description']

## Issues: 
1. Kernal, when re-run provides results, but stops before then
2. Using pd.to_csv and pd.read_csv file does not merge, but when runnng all kernals, it works fine
3. Merge isn't working still...
4. 